<a href="https://colab.research.google.com/github/tunde99/masakhane-mt/blob/master/ha_en_v3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g5fFZcDVriFGmQEVUUBfkb-M-nrXirogq2cnFu5mv7mQOC5sHK3vqc
Mounted at /content/drive


In [14]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ha" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline_v3.0" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (target_language, source_language, tag)
# create a folder to store model files
!mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag/models/${tgt}${src}_reverse_transformer/"

In [15]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/ha-en-baseline_v3.0


In [16]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 7.3MB/s 


In [17]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/en-ha.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/en.zip
  24 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/ha.zip

 289 MB Total size
./JW300_latest_xml_en-ha.xml.gz ... 100% of 3 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ha.zip ... 100% of 24 MB


In [18]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

# Uncomment the following lines of code if your language pair is part of the global test set
# else, create the test set and upload them to this notebook.



--2021-05-06 23:46:37--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.01s   

2021-05-06 23:46:38 (18.4 MB/s) - ‘test.en-any.en’ saved [277791/277791]



In [ ]:
# If you face error after running this cell, it means that your language pair is not part of the global test set.

# 1. Comment the following lines of code.
# 2. Create the test set 
# 3. upload test set to this notebook ---> run the next cell to upload 

# ===> Go to the github for starter note on how to create global test set.<===

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

In [19]:
# Load created test sets
from google.colab import files
upload_files = files.upload()

Saving test.en to test.en
Saving test.ha to test.ha


In [20]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [21]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 5417/237064 lines since contained in test set.


,source_sentence,target_sentence
0,Questions From Readers,Tambayoyi Daga Masu Karatu
1,How seriously should Christians view an engage...,Yaya ya kamata Kiristoci su ɗauki riƙo ?
2,An engagement to marry is a cause for happines...,"Riƙo dalili ne na farinciki , amma kuma muhimm..."
3,No mature Christian should take an engagement ...,Kada wani riƙaƙƙen Kirista ya yi wasa da riƙo ...
4,The period of engagement is also a time for th...,Lokacin riƙo lokaci ne da mutane biyun za su s...
5,"In discussing this topic , we need to realize ...","Cikin tattauna wannan zance , ya kamata mu gan..."
6,The Bible illustrates this .,Littafi Mai - Tsarki ya bada misalin wannan .
7,"Lot’s two daughters , who had “ never had inte...","’ Ya’ya mata biyu na Lutu , waɗanda “ ba su sa..."
8,Lot’s ‘ sons - in - law were to take his daugh...,"‘ Surukan Lutu za su aure ’ ya’yansa mata , ’ ..."
9,Were the daughters adults ?,’ Ya’yansa mata sun yi girma kuwa ?


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [22]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149827 sha256=010e78d0f48dd6727f824c382ce5084f2e550af2ced499bd3fcf72fe6491180d
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [24]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
“ Preaching in a territory where you meet individuals who are so eager to learn the truth that they want to study the Bible with you every day is such a joy , ” says Stephanie .
Where a public reading was forbidden , some sent copies to every member of the church . They wanted no further dealings with false religion !
How can couples apply Ephesians 4 : 26 , 27 in their marriage ?
Why may we look to the future with confidence ?
However , she did what she could , and this can teach us a lesson .
Consider what happened just before his arrest , trial , and execution .
Many overcame their superstitious fears , and the number attending grew to over 80 .
This spectacular display attracts tourists from all over the country .
Therefore , beloved ones , since you are awaiting these things , do your utmost to be found finally by him spotless and unblemished and in peace . ”
God’s Word , the Bible , foretold that in our time people would be “ self - assuming , haughty . ”

==> tr



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [35]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.8.0.
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
  Using cached https://files.pythonhosted.org/packages/94/99/5861239a6e1ffe66e120f114a4d67e96e5c4b17c1a785dfc6ca6769585fc/torch-1.8.0-cp37-cp37m-manylinux1_x86_64.whl
  Created wheel for joeynmt: filename=joeynmt-1.3-cp37-none-any.whl size=84842 sha256=5571f76e4cad777f69731f3df34524ca2f30e8cde4c0c2e575a504ab9c3e0b89
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucahxhv6/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
Successfully built joeynmt
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: joeynmt 1.3
    Uninstalling joeynmt-1.3:
      Successfully uninstalled joeynmt-1.3
Looking in links: https://download.pytorch.org/whl/torch_st

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [36]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE Hausa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.ha     test.ha	   train.en
dev.bpe.en	dev.ha	     test.en	     train.bpe.en  train.ha
dev.bpe.ha	test.bpe.en  test.en-any.en  train.bpe.ha  vocab.txt
bpe.codes.4000	dev.en	test.bpe.en  test.en-any.en  train.bpe.ha
dev.bpe.en	dev.ha	test.bpe.ha  test.ha	     train.en
dev.bpe.ha	models	test.en      train.bpe.en    train.ha
BPE Hausa Sentences
Kada mu zama kamar Ab@@ ner da Ab@@ sal@@ om da suka ci am@@ ana .
[ 1 ] ( sa@@ kin la@@ yi na 7 ) An canja wasu sun@@ a@@ ye .
Idan muna da aminci da kuma tawali’u , za mu mai da hankali ga abin da Jehobah yake so mu yi , ba abin da muke so ba
Idan mu masu aminci ne da kuma kirki , za mu riƙa taimaka wa mutane , kuma za mu bi da su yadda Jehobah yake so
Idan muna da aminci da kuma ƙarfin zuciya , za mu yi biyayya ga Jehobah ko da yin hakan yana da wuya ko kuma muna far@@ gabaCombined BPE Vocab
ʺ
tabl@@
ö
£
ş
Ó@@
lima
refore
Ž@@
↓


In [37]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	test.bpe.en  test.en-any.en  train.bpe.ha
dev.bpe.en	dev.ha	test.bpe.ha  test.ha	     train.en
dev.bpe.ha	models	test.en      train.bpe.en    train.ha


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [38]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 60                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [39]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-05-07 00:12:55,070 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-07 00:12:55,137 - INFO - joeynmt.data - Loading training data...
2021-05-07 00:12:58,539 - INFO - joeynmt.data - Building vocabulary...
2021-05-07 00:12:58,809 - INFO - joeynmt.data - Loading dev data...
2021-05-07 00:12:58,835 - INFO - joeynmt.data - Loading test data...
2021-05-07 00:12:59,522 - INFO - joeynmt.data - Data loaded.
2021-05-07 00:12:59,522 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-07 00:12:59,723 - INFO - joeynmt.model - Enc-dec model built.
2021-05-07 00:13:00.812402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-07 00:13:04,459 - INFO - joeynmt.training - Total params: 12152320
2021-05-07 00:13:07,805 - INFO - joeynmt.helpers - cfg.name                           : haen_reverse_transformer
2021-05-07 00:13:07,805 - INFO - joeynmt.helpers - cfg.data.src                       : h

In [40]:
##############################################################################################################################################################
# Copy the created models from the notebook storage to google drive for persistant storage 
# If you face error like this: cp: target '/content/drive/My Drive/masakhane/ha-en-baseline_v2.0/models/enha_reverse_transformer/' is not a directory, then
# comment the active line after, uncomment the line below, run the cell again.
##############################################################################################################################################################
# !cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${src}${tgt}_reverse_transformer/"
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${tgt}${src}_reverse_transformer/"

In [41]:
# Output our validation accuracy
! cat "$gdrive_path/models/${tgt}${src}_reverse_transformer/validations.txt"

Steps: 1000	Loss: 107483.45312	PPL: 74.86884	bleu: 0.60422	LR: 0.00030000	*
Steps: 2000	Loss: 93090.47656	PPL: 42.00643	bleu: 3.33211	LR: 0.00030000	*
Steps: 3000	Loss: 82107.88281	PPL: 27.02719	bleu: 4.50690	LR: 0.00030000	*
Steps: 4000	Loss: 75834.29688	PPL: 21.00883	bleu: 6.85707	LR: 0.00030000	*
Steps: 5000	Loss: 71199.51562	PPL: 17.44136	bleu: 9.29269	LR: 0.00030000	*
Steps: 6000	Loss: 67609.32812	PPL: 15.09991	bleu: 11.56016	LR: 0.00030000	*
Steps: 7000	Loss: 65081.41016	PPL: 13.64246	bleu: 13.96047	LR: 0.00030000	*
Steps: 8000	Loss: 62742.10156	PPL: 12.41937	bleu: 15.80832	LR: 0.00030000	*
Steps: 9000	Loss: 60585.40625	PPL: 11.38914	bleu: 17.56147	LR: 0.00030000	*
Steps: 10000	Loss: 59042.35156	PPL: 10.70491	bleu: 18.05234	LR: 0.00030000	*
Steps: 11000	Loss: 57884.32812	PPL: 10.21856	bleu: 19.13852	LR: 0.00030000	*
Steps: 12000	Loss: 56737.23828	PPL: 9.75858	bleu: 19.02220	LR: 0.00030000	*
Steps: 13000	Loss: 55554.98438	PPL: 9.30615	bleu: 20.26899	LR: 0.00030000	*
Steps: 14000	L

In [42]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${tgt}${src}_reverse_transformer/config.yaml"

2021-05-07 04:02:22,372 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-07 04:02:22,373 - INFO - joeynmt.data - Building vocabulary...
2021-05-07 04:02:22,650 - INFO - joeynmt.data - Loading dev data...
2021-05-07 04:02:22,663 - INFO - joeynmt.data - Loading test data...
2021-05-07 04:02:22,698 - INFO - joeynmt.data - Data loaded.
2021-05-07 04:02:22,737 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-05-07 04:02:26,131 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-07 04:02:26,348 - INFO - joeynmt.model - Enc-dec model built.
2021-05-07 04:02:26,422 - INFO - joeynmt.prediction - Decoding on dev set (data/haen/dev.bpe.en)...
2021-05-07 04:02:36,644 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-05-07 04:02:36,644 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-05-07 04:02:36,645 - WARNIN

# Credits

Thank you Cate, for helping to debug.

Thank you joeynmt team, for joeynmt

Thank you Masakhane github team, for the starter notebook.

Author: Tunde Ajayi